Table of contents: TBA

## Removal of giveaway posts

a) __Naive Bayes classification__ of FB posts to detect viral marketing.  
b) __remove whole threads__ that started with a giveaway post. 

Before removal: 114,826 documents  
After removal: 59,207 documents

TODO: hide code

In [ ]:
import pandas as pd

from src.giveaway.GiveawayClassifier import GiveawayClassifier
from src.utility.general import export_serialized

In [ ]:
# load in dataset you with to work on
df = pd.read_csv(
    'data/hpv_data_reactions_copy.csv',
    parse_dates = ['time']
)

Load training data for the classifier (494 documents).  

POST-level content found to contain Marie Louise's stopwords.  
Hand labeled by one person.

In [ ]:
labeled = (pd.read_csv('data/200414_giveaway_training.csv')
           # drops 2 rows with a missing label (496 rows in original file)
           .dropna(subset=['giveaway']))

X = labeled['text']
y = labeled['giveaway']

Train the Giveaway Classifier.

In [ ]:
gc = GiveawayClassifier(X=X, y=y)
gc.train()
gc.report

Classify only POST-level content in the loaded dataset.  
The model classifies short comments unreliably.

In [ ]:
df_post = df.query('content_type == "POST"')

giveawas_df = (gc
               .predict_new(df_post.text, negative_for_url=True)
               .query('predicted == 1')
               .rename(columns={'index': 'id_orig'})
              )

giveawas_df

Filter found threads from the original dataset  
a) find post_id's that were labeled as a giveaway  
b) filter threads with such post ids out  

In [ ]:
bad_threads = df.query('@giveawas_df.id_orig').post_id
bad_threads = [num for num in bad_threads]

# remove bad threads
S1_giveaway_removed = df.query('post_id != @bad_threads')

# save whole dataframe
S1_giveaway_removed.to_csv('data/S1_giveaway_removed.csv')

# save texts with ID
export_serialized(
    df=S1_giveaway_removed,
    column='text',
    path='data/S1_fb_texts.ndjson'
)

<br>

## Preprocessing
_[text_to_x](https://github.com/centre-for-humanities-computing/text_to_x)_

a) __tokens__, __lemmas__, __POS__ & __dependency parsing__ using [Stanza](https://github.com/stanfordnlp/stanza)  
b) __NER__ using [Flair](https://github.com/flairNLP/flair)

Takes a lot of time to run. 
It is recommended that you run this part from the terminal.

```bash
cd hpv-vaccine
python3 src/preprocessing.py -p data/S1_fb_texts.ndjson -o data/S2_fb_prep.ndjson --lang 'da' --jobs 4
```


<br>

## Feature selection 

a) __Filter out non-meaningful Parts of Speech from all texts__.   
Only NOUN, PROP-NOUN, ADJ, VERB and ADVERB will be kept


b) __Neural detection of phrases__.  
If two tokens appear together often, they will be concatenated into a single token.

In [ ]:
import ndjson

from src.utility import phraser
from src.utility.general import load_data

In [ ]:
# import preprocessed data
texts_id = load_data('data/S3_prep_NO5.ndjson')

# phraser has both a) & b) functionality
texts_phrased = phraser.train(
    texts_id,
    lang='da',
    out_path='data/S3_fb_phrase.ndjson'
)

# texts only
texts = [doc['text'] for doc in texts_phrased]

<br>

## Seed selection

a) __Train a CBOW model__  
To be used for finding related words to query.  
Intentions behind the parameters:
- words that appear together in the whole FB post (window=20)
- frequent words, so that the seeds are generalizable (min_count=100)

_comment: potentially this could be taken care of by PmiSvdEmbeddings._

b) __Enhance phrase list__  
Add synonyms and related words to a given phrase list. This will be used as guide the topic model.

In [ ]:
from gensim.models import Word2Vec, KeyedVectors

# from src.embeddings.pmisvd import PmiSvdEmbeddings
from src.embeddings.query_ops import import_query, get_related

In [ ]:
# import phrase list
query_list = import_query(
    ordlist_path='data/200729_hpv_query.csv',
    lang='da',
    col='term'
)

# train a cbow model
cbow_texts = Word2Vec(
    texts,
    size=100, window=20, min_count=100,
    sg=0, hs=0,
    iter=500, workers=4
)

# get a list of words similar to those in the phrase list
query_related = get_related(cbow_texts.wv, query_list, topn=10, cutoff=50)

# save + display a sample
(query_related
 .to_csv('data/S4_query_related.csv'))

Now the seeds have to be __manually redacted__.

<br>

## Topic modeling

In [ ]:
from src.lda.asymmetric import grid_search_lda_ASM
from src.lda.seeded import grid_search_lda_SED
from src.utility.general import compile_report

In [ ]:
# extract topic seeds
# HIDE IT SOMEWHERE
S5_query_redacted = pd.read_csv('data/S5_query_redacted.csv')
seeds = (S5_query_redacted
         .dropna(subset=['seed'])
         .groupby('topic')['seed']
         .apply(list)
         .to_frame()
         .seed
         .tolist())

In [ ]:
grid_search_lda_SED(
    texts=texts,
    seed_topic_list=[], #feed
    n_topics_range=[5, 10, 15, 20, 25, 30],
    priors_range=[(0.1, 0.01), (0.5, 0.1)], #feed
    out_dir='models/200730_seed_no5/',
    n_top_words=10,
    vectorizer_type='count',
    iterations=500,
    save_doc_top=True,
    verbose=False
)

In [ ]:
compile_report('models/test_seeded/report_lines/')

In [ ]:
grid_search_lda_ASM(
    texts=texts,
    n_topics_range=[5, 10, 15, 20, 25, 30],
    iterations=500,
    passes=2,
    out_dir='models/200730_asm_no5/',
    verbose=False,
    save_doc_top=True,
)

In [ ]:
compile_report('models/200730_asm_no5/report_lines/')

<br>

## Model evolution

In [ ]:
import src.topicevolution.run_ntr as ntr 

In [ ]:
import ndjson
with open('models/200730_asm_no5/doctop_mats/10T_ASM_mat.ndjson') as f:
    doctop = ndjson.load(f)

In [ ]:
import pandas as pd
dt_df = pd.DataFrame(doctop)

In [ ]:
dt_df.to_csv('10T_ASM_mat.csv')

In [ ]:
ntr.kz(dt_df[0], window=3, iterations=2)

In [ ]:
with open('data/S1_fb_texts.ndjson') as f:
    tid = ndjson.load(f)

In [ ]:
ids = [doc['ID'] for doc in tid]

In [ ]:
ntr.calculate_ntr(
    doc_top_prob=doctop,
    ID=range(49892),
    window=[10, 50, 100],
    out_dir='models/200730_asm_no5/10T_ASM/'
)

<br>

## Topic usage